In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

from numpy import interp
from scipy.stats import boxcox

from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import f1_score, make_scorer, auc, average_precision_score, precision_score, roc_curve, precision_recall_curve, recall_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Check raw data
raw_data = pd.read_csv("/kaggle/input/cervical-cancer-risk-classification/kag_risk_factors_cervical_cancer.csv")
raw_data.head(7)

# Find the number of missing entries in each column
data_with_null =raw_data.replace('?', np.nan)
data_with_null.isnull().sum() 

# Remove the two columns with 787 missing entries
sparsity_removed_data = data_with_null.drop(columns = ['STDs: Time since first diagnosis', 'STDs: Time since last diagnosis'])
sparsity_removed_data.shape

# Count number of missing entries in each row
missing_entries = sparsity_removed_data.isnull().sum(axis=1).tolist()

# Group rows with the same number of missing entries
null_mapping = dict((x, missing_entries.count(x)) for x in missing_entries)
sorted(null_mapping.items())

# Remove all the rows with any null entries
null_removed_data = sparsity_removed_data.dropna()
null_removed_data.isnull().sum()

# All categorical columns
object_cols = [col for col in null_removed_data.columns if null_removed_data[col].dtype == "object"]

# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: null_removed_data[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

# Convert all categorical columns to float data
column_new_dtypes = {key: 'float64' for key in d}
df = null_removed_data.astype(column_new_dtypes)
df.info()

#df = null_removed_data.apply(pd.to_numeric)
#df.info()

# Get details of the null removed data, i.e., Mean, Min, Max etc
df.describe()

In [ ]:
# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
# Correlation Matrix for each features
corrmat = round(df.corr(), 2)
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,fmt='.2f',cmap="YlGnBu")

In [ ]:
# BoxPlot showing raw distribution of features
plt.figure(figsize=(16,4))
df.iloc[:,:].boxplot()
plt.title('(Raw) Distribution of Features', fontsize=17)
# Get current axes
ax = plt.gca()
# Make space for and rotate the x-axis tick labels
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.show()

In [ ]:
df.mode()

In [ ]:
df.median()

In [ ]:
# Remove outliers
df.drop(df.index[df['Age'] > 52], inplace = True)
df.drop(df.index[df['Number of sexual partners'] > 8], inplace = True)

In [ ]:

# Define target variables and remove target columns from data
y1 = df.Hinselmann
y2 = df.Schiller
y3 = df.Citology
y4 = df.Biopsy
X = df.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis=1)

# Count number of class 1 and class 0 samples for each target
print('_' * 30)
print("Capturing Class Imbalance")
print(y1.value_counts())
print(y2.value_counts())
print(y3.value_counts())
print(y4.value_counts())

t1 = sparsity_removed_data.Hinselmann
t2 = sparsity_removed_data.Schiller
t3 = sparsity_removed_data.Citology
t4 = sparsity_removed_data.Biopsy

# Count number of class 1 and class 0 samples for each target
print('_' * 30)
print("Data with null values:")
print(t1.value_counts())
print(t2.value_counts())
print(t3.value_counts())
print(t4.value_counts())


In [ ]:
# Convert age to Age groups - Min age: 13, Max age: 52
# 10 to 14 --> Group 1
# 15 to 19 --> Group 2
# ...
# 50 to 54 --> Group 9

age_bins = np.arange(9, 55 , 5)
age_labels = np.arange(1, 10)
X['age_group'] = pd.cut(X.Age, bins = age_bins, labels = age_labels)
X['age_group'] =  pd.to_numeric(X['age_group'])

# df[['Age','age_group']].head(20)

In [ ]:
# Similarly convert first sexual intercourse age, Min age: 10, Max age: 32
fsi_bins = np.arange(9, 35 , 5)
fsi_labels = np.arange(1, 6)
X['fsi_group'] = pd.cut(X['First sexual intercourse'], bins = fsi_bins, labels = fsi_labels)
X['fsi_group'] =  pd.to_numeric(X['fsi_group'])

# df[['Age', 'First sexual intercourse', 'fsi_group']].head(20)

In [ ]:
# Drop Age, First Sexual Intercourse columns
X = X.drop(columns=['Age', 'First sexual intercourse'], axis=1)

In [ ]:
X.nunique(axis=0).sort_values()

In [ ]:
# Remove columns with only one unique value
X = X.drop(['STDs:cervical condylomatosis', 'STDs:AIDS'], axis=1)

In [ ]:
# sns.relplot(x="Number of sexual partners", y=y1.name, data=df);

In [ ]:
from sklearn.feature_selection import mutual_info_classif

"""def make_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y1)
positive_mi = mi_scores[mi_scores != 0]
positive_mi

"""


In [ ]:
"""def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(positive_mi)"""

In [ ]:
"""norm_columns = ['Age', 'Number of sexual partners', 'First sexual intercourse', 
                'Num of pregnancies', 'Smokes (years)', 'Smokes (packs/year)', 
               'Hormonal Contraceptives (years)', 'IUD (years)', 'STDs (number)',
               'STDs: Number of diagnosis']

X.hist(figsize=(15,15))"""

In [ ]:
# Normalise Age
"""X['Age Norm'] = boxcox(X['Age'], 0.5)
X.hist('Age Norm')"""

In [ ]:
from scipy.special import boxcox1p

# Normalise Age
"""X['Num of pregnancies (Norm)'] = boxcox1p(X['Num of pregnancies'], 0.7)
X.hist('Num of pregnancies (Norm)')"""

In [ ]:
# First regularize data b/w 1 and 2
scaler = MinMaxScaler(feature_range=(1, 2))

# Use PowerTransformer to normalize data
power = PowerTransformer(method='box-cox')
pipeline = Pipeline(steps=[('s', scaler),('p', power)])
Xnorm = pipeline.fit_transform(X)

Xnorm = pd.DataFrame(Xnorm)
Xnorm.hist(figsize=(15,15))
#result_df = df1.apply(lambda col: stats.boxcox(col, a.loc[col.name]))

In [ ]:
Xnorm.columns = X.columns

In [ ]:
X_columns = X.columns.tolist()
X_columns

In [ ]:
"""#feat_name = 'STDs:condylomatosis'
for ft_name in X_columns:

    # Frequency distribution of Age
    print(X.groupby([ft_name]).size())
    
    for tes in [y1.name, y2.name, y3.name, y4.name]:
        print(tes)
        hins_age = df.loc[df[tes] == 1]
        print(hins_age.groupby([ft_name]).size())"""

In [ ]:
"""for tes in [y1.name, y2.name, y3.name, y4.name]:
    print(tes)
    hins_age = df.loc[df[tes] == 1]
    hins_age.groupby([feat_name]).size()"""

In [ ]:
# BoxPlot showing raw distribution of features
plt.figure(figsize=(16,4))
X.iloc[:,:].boxplot()
plt.title('(Raw) Distribution of Features - Transformed', fontsize=17)
# Get current axes
ax = plt.gca()
# Make space for and rotate the x-axis tick labels
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.show()

One of the most important step while working with highly imbalanced dataset is to balance the classes. There are several tecniques which provide a mechanism for balancing the dataset. These involve Random undersampling of the majority class, Oversampling of the minority class, Cluster based Oversampling and Synthetic Minority Oversampling Technique (SMOTE). Here we will use SMOTE.

In [ ]:
# Balance Classes
# Join the train data
train1 = Xnorm.join(y1)
train2 = Xnorm.join(y2)
train3 = Xnorm.join(y3)
train4 = Xnorm.join(y4)

#print('Data (Hinselmann) shape before balancing:',len(train1))
print('\nCounts of positive VS negative (Hinselmann) in original data:')
print(train1.Hinselmann.value_counts())
print('-'*40)

#print('Data (Schiller) shape before balancing:',train2.shape)
print('\nCounts of positive VS negative (Schiller) in original data:')
print(train2.Schiller.value_counts())
print('-'*40)

#print('Data (Citology) shape before balancing:',train3.shape)
print('\nCounts of positive VS negative (Citology) in original data:')
print(train3.Citology.value_counts())
print('-'*40)

#print('Data (Biopsy) shape before balancing:',train4.shape)
print('\nCounts of positive VS negative (Biopsy) in original data:')
print(train4.Biopsy.value_counts())
print('-'*40)

In [ ]:
y1.name

In [ ]:
# Oversample positives. Imblearn's ADASYN was built for class-imbalanced datasets
X1_bal, y1_bal = ADASYN(sampling_strategy='minority',random_state=0).fit_resample(
    Xnorm,
    y1)

# Oversample positives. Imblearn's ADASYN was built for class-imbalanced datasets
X2_bal, y2_bal = ADASYN(sampling_strategy='minority',random_state=0).fit_resample(
    Xnorm,
    y2)

# Oversample positives. Imblearn's ADASYN was built for class-imbalanced datasets
X3_bal, y3_bal = ADASYN(sampling_strategy='minority',random_state=0).fit_resample(
    Xnorm,
    y3)

# Oversample positives. Imblearn's ADASYN was built for class-imbalanced datasets
X4_bal, y4_bal = ADASYN(sampling_strategy='minority',random_state=0).fit_resample(
    Xnorm,
    y4)

# Join X and y
X1_bal = pd.DataFrame(X1_bal,columns=Xnorm.columns)
y1_bal = pd.DataFrame(y1_bal,columns=['Hinselmann'])
balanced1 = X1_bal.join(y1_bal)

# Join X and y
X2_bal = pd.DataFrame(X2_bal,columns=Xnorm.columns)
y2_bal = pd.DataFrame(y2_bal,columns=['Schiller'])
balanced2 = X2_bal.join(y2_bal)

# Join X and y
X3_bal = pd.DataFrame(X3_bal,columns=Xnorm.columns)
y3_bal = pd.DataFrame(y3_bal,columns=['Citology'])
balanced3 = X3_bal.join(y3_bal)

# Join X and y
X4_bal = pd.DataFrame(X4_bal,columns=Xnorm.columns)
y4_bal = pd.DataFrame(y4_bal,columns=['Biopsy'])
balanced4 = X4_bal.join(y4_bal)


print('-'*40)
print('Data (Hinselmann) shape after balancing:',balanced1.shape)
print('\nCounts of positive VS negative in new data:')
print(balanced1.Hinselmann.value_counts())

print('-'*40)
print('Data (Schiller) shape after balancing:',balanced2.shape)
print('\nCounts of positive VS negative in new data:')
print(balanced2.Schiller.value_counts())

print('-'*40)
print('Data (Citology) shape after balancing:',balanced3.shape)
print('\nCounts of positive VS negative in new data:')
print(balanced3.Citology.value_counts())

print('-'*40)
print('Data (Biopsy) shape after balancing:',balanced4.shape)
print('\nCounts of positive VS negative in new data:')
print(balanced4.Biopsy.value_counts())

In [ ]:
y1_bal.columns.name = 'Hinselmann'
y2_bal.columns.name = 'Schiller'
y3_bal.columns.name = 'Citology'
y4_bal.columns.name = 'Biopsy'

targets = [y1_bal.columns.name, y2_bal.columns.name, y3_bal.columns.name, y4_bal.columns.name]

In [ ]:

def draw_cv_roc_curve(classifier, cv, X, y, title='ROC Curve'):
    """
    Draw a Cross Validated ROC Curve.
    Keyword Args:
        classifier: Classifier Object
        cv: StratifiedKFold Object
        X: Feature Pandas DataFrame
        y: Response Pandas Series
    Example largely taken from http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
    """
    # Creating ROC Curve with Cross Validation
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    mean_auc = 0
    std_auc = 0

    i = 0
    for train, test in cv.split(X, y):
        
        probas_ = classifier.fit(X.iloc[train], y.iloc[train]).predict_proba(X.iloc[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))

        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

        i += 1
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()
    return mean_auc


In [ ]:
def draw_cv_pr_curve(classifier, cv, X, y, title='PR Curve'):
    """
    Draw a Cross Validated PR Curve.
    Keyword Args:
        classifier: Classifier Object
        cv: StratifiedKFold Object: (https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation)
        X: Feature Pandas DataFrame
        y: Response Pandas Series

    Largely taken from: https://stackoverflow.com/questions/29656550/how-to-plot-pr-curve-over-10-folds-of-cross-validation-in-scikit-learn
    """
    y_real = []
    y_proba = []

    i = 0
    for train, test in cv.split(X, y):
        trainer = classifier.fit(X.iloc[train], y.iloc[train])
        probas_ = trainer.predict_proba(X.iloc[test])
        
        #plot_importance(xgbc)
        #print(get_feature_importance(trainer))
        
        # Compute ROC curve and area the curve
        precision, recall, _ = precision_recall_curve(y.iloc[test], probas_[:, 1])
        
        # Plotting each individual PR Curve
        plt.plot(recall, precision, lw=1, alpha=0.3,
                 label='PR fold %d (AUC = %0.2f)' % (i, average_precision_score(y.iloc[test], probas_[:, 1])))

        y_real.append(y.iloc[test])
        y_proba.append(probas_[:, 1])

        i += 1

    y_real = np.concatenate(y_real)
    y_proba = np.concatenate(y_proba)

    precision, recall, _ = precision_recall_curve(y_real, y_proba)

    pr_auc = average_precision_score(y_real, y_proba)
    plt.plot(recall, precision, color='b',
             label=r'Precision-Recall (AUC = %0.2f)' % (pr_auc),
             lw=2, alpha=.8)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()
    return pr_auc

In [ ]:
# Setting up simple RF Classifier
rfc = RandomForestClassifier(random_state=42)

xgbc = XGBClassifier(n_estimators=10, 
                     max_depth=5, 
                     learning_rate=0.4, 
                     random_state=42)

svc = Pipeline([ ("pre", preprocessing.StandardScaler()), 
                    ("classifier", SVC(C=1, 
                                       probability=True, random_state=42))])

knn = KNeighborsClassifier()

clfs = [svc, rfc, knn, xgbc]

In [ ]:
cvs = []
for split_value in [4, 5, 6, 7, 8, 9]:
    # Set up Stratified K Fold
    cvs.append(StratifiedKFold(n_splits=split_value, random_state=0))


In [ ]:
cvs[1].n_splits

In [ ]:
def get_classifier_name(clf):
    cn = clf.__class__.__name__
    class_name = cn if (cn != 'Pipeline') else 'SVC'
    return class_name

In [ ]:
oversampled_data = [(X1_bal, y1_bal), (X2_bal, y2_bal), (X3_bal, y3_bal), (X4_bal, y4_bal)]
mean_scores_roc = {}
for clf in clfs:
    for cv in cvs:
        clf_name = get_classifier_name(clf)
        key = clf_name+str(cv.n_splits)
        #Add new keys with classifier name in the dictionary
        mean_scores_roc[key] = []
        print('_'*30,'\n\n',clf_name,'\n', '_'*30)
        for X_bal,target in oversampled_data:
            roc_title = str(cv.n_splits) + '-fold Cross Validated ROC curve for ' + target.columns.name + ' - ' + clf_name
            mean_scores_roc[key].append(draw_cv_roc_curve(clf, cv, X_bal, target, title=roc_title))

In [ ]:
mean_scores_pr = {}
for clf in clfs:
    
    for cv in cvs:
        clf_name = get_classifier_name(clf)
        key = clf_name+str(cv.n_splits)
        
        mean_scores_pr[key] = []
        print('_'*30,'\n\n',clf_name,'\n', '_'*30)
        for X_bal,target in oversampled_data:
            pr_title = str(cv.n_splits) + '-fold Cross Validated PR Curve for ' + target.columns.name + ' - ' + clf_name
            mean_scores_pr[key].append(draw_cv_pr_curve(clf, cv, X_bal, target, title=pr_title))

In [ ]:
# DataFrame to store classifier performance
performance_roc = pd.DataFrame(columns=targets)
for clf in clfs:
    for cv in cvs:
        key = get_classifier_name(clf)+str(cv.n_splits)
        performance_roc.loc[key,
                            targets] = mean_scores_roc[key]
print('_'*50,'\n\n', 'Table depicting ROC AUC scores', '\n', '_'*50)
    
performance_roc

In [ ]:
# DataFrame to store classifier performance
performance_roc = pd.DataFrame(columns=targets)
for clf in clfs:
    for cv in cvs:
        key = get_classifier_name(clf)+str(cv.n_splits)
        performance_roc.loc[key,
                            targets] = mean_scores_roc[key]
print('_'*50,'\n\n', 'Table depicting ROC AUC scores', '\n', '_'*50)
    
performance_roc

In [ ]:
# DataFrame to store classifier performance
performance_pr = pd.DataFrame(columns=targets)

for clf in clfs:
    for cv in cvs:
        key = get_classifier_name(clf)+str(cv.n_splits)
        performance_pr.loc[key,
                            targets] = mean_scores_pr[key]
print('_'*50,'\n\n', 'Table depicting Precision-Recall AUC scores', '\n', '_'*50)
    
performance_pr

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from IPython.display import display

# Get feature names to be displayed
feat_names = X1_bal.columns.tolist()

# Select each classifier one by one
for clf in clfs:
    
    print('_'*40,'\n',get_classifier_name(clf),'\n', '_'*40)
    # PermutationImportance object for each classifier
    permuter = PermutationImportance(clf, scoring='roc_auc', cv='prefit', random_state=42)
    
    # Calculate feature importance for all 4 target variables
    for X_bal,y_bal in oversampled_data:
        permuter.fit(X_bal, y_bal)
        print('_'*40,'\n\n','Feature Importance (', y_bal.columns.name, ') - ', get_classifier_name(clf), '\n', '_'*40,'\n')
        display(eli5.show_weights(permuter, feature_names = feat_names))